# Ingeniería de variables

30 de mayo de 2023

**Este código no se puede ejecutar porque los datos que contienen
los números de documento de los estudiantes no están compartidos
en este repositorio público.**

Nuevas variables:

| variable | descripción |
| --- | ---|
| extranjero  | si DNI > 90millones |
| curso | las comsiones se repiten |
| turno | A,B,C,D |
| n_alum | número de estudiantes en el curso |
| p_ext | porcentaje de extranjeros en el curso |
| recurso | cantidad de veces que se inscribió anteriormente (sin distinguir materia) |
| p_recursa | porcentaje de recursantes en el curso |
| sala | id de aula |
| pa1_prom | promedio de notas de parcial 1 en el curso |
| pa2_prom | promedio de notas de parcial 2 en el curso |
| final_prom | promedio de notas de final en el curso |
| edad        | estimada con dni | 
| prom_edad   | promedio de la variable edad en el curso | 
| condición | Abandona1, Abandona2, Insuficiente, Examen, Promociona (solo parciales)|
| abandona1_p | porcentaje en condición Abandona1 del curso |
| abandona2_p | porcentaje en condición Abandona2 del curso, sobre los que rindieron parcial 1 |
| valido1 | 1: válido, 0: no se cargaron notas de parcial 1 en este curso | 
| valido2 | 1: válido, 0: no se cargaron notas de parcial 2 en este curso | 


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../datos/dataset_01.csv")
print("Cantidad de registros:",len(df))

Cantidad de registros: 233615


In [2]:
################################################
#
# Imputación DNIs
#
# para poder asignar un grupo de edad.
#
################################################

# Cambio DNI por número de estudiante
df['estudiante'] = df.groupby(by='dni').ngroup()

# Nueva variable: extranjero
df.loc[(df["dni"] >= 90000000), "extranjero"] = 1
df.loc[(df["dni"] < 90000000), "extranjero"] = 0
df['extranjero'] = df['extranjero'].values.astype(int)


# No uso el siguiente enfoque, lo mantengo como referencia.
# Agrega un grupo_dni para estimar la edad:
# el dni del grupo n se encuentra entre n*5000000 y (n+1)*5000000
# df['grupo_dni'] = df['dni'].apply(lambda x: x // 5000000)

#
# Imputación de DNIs
#
# El número de extranjeros es grande, modifico sus dni con una distribución
# igual a la de los dni no extranjeros, tomando un elemento al azar entre
# los dni de cada cuatrimestre. Incluyo a los dni menores a 1 millón.

# Creo una categoría por anio-cuat
df['cuatrimestre'] = df["anio"].astype(str) + "-" + df["cuat"].astype(str)

cuatrimestres = df['cuatrimestre'].unique()
for c in cuatrimestres:
    values = df.loc[((df["dni"] < 90000000) & (df["dni"] > 1000000)) & (df['cuatrimestre']==c)]['dni']
    cant = len(df.loc[((df["dni"] > 90000000) | (df["dni"] < 1000000)) & (df['cuatrimestre']==c)])
    nuevos = np.random.choice(values, cant)
    df.loc[((df["dni"] > 90000000) | (df["dni"] < 1000000)) & (df['cuatrimestre']==c), "dni"] = nuevos

In [3]:
################################################
#
# Otras variables
#
################################################

# curso: comision se repite en cada cuatrimestre, lo cambio por un id único de curso
df['curso'] = df.groupby(by=['cuatrimestre','COMISION']).ngroup()
df['curso'] = df['curso'].values.astype(int)
df = df.drop(['COMISION'], axis=1)

# turno : según el código de horario A: muy temprano, B: media mañana, C: tarde, D:noche
df.loc[df['HORARIO'].isin([651,681,621,550,610,601,600,261,201,351,321]), 'turno'] = 'A'
df.loc[df['HORARIO'].isin([652,682,622,623,653,643,693,213,612,553,662,262,633,523,692,613,602]), 'turno'] = 'B'
df.loc[df['HORARIO'].isin([656,657,626,627,616,607,696,637,654,655,605,645,694,556,635,644,
                           546,544,615,235,204,205,245,614,636,526,625,606,634,244,234]), 'turno'] = 'C'
df.loc[df['HORARIO'].isin([658,628,558,619,639,]), 'turno'] = 'D'
df = df.drop(['HORARIO'], axis=1)

# Número de estudiantes por curso
df['n_alum'] = df.groupby('curso')['estudiante'].transform('size')
df['n_alum'] = df['n_alum'].values.astype(int)

# Porcentaje de extranjeros en el curso
df['sumext'] = df.groupby(['curso'])['extranjero'].transform('sum')
df['p_ext'] = df['sumext']/df['n_alum']
df = df.drop(['sumext'], axis=1)

# Cantidad de veces que recursó
df['temp'] = np.ones(len(df))
df['recurso'] = df.groupby('estudiante')['temp'].transform('cumsum')
df['recurso'] = df['recurso'] - df['temp']
df['recurso'] = df['recurso'].values.astype(int)
df = df.drop(['temp'], axis=1)

# Porcentaje de recursantes en el curso
df.loc[df['recurso'] > 0, 'temp'] = 1
df['nrecursantes'] = df.groupby(['curso'])['temp'].transform('sum')
df['p_recursa'] = df['nrecursantes'] / df['n_alum']
df = df.drop(['temp'], axis=1)
df = df.drop(['nrecursantes'], axis=1)

# Un id para cada aula, los números de aula se pueden repetir en distintas sedes
df['sala'] = df.groupby(by=['SEDE','AULA']).ngroup()
df = df.drop(['AULA'], axis=1)

# Promedio de notas de parcial 1, parcial 2 y final por curso.
df['pa1_prom'] = df.groupby(['curso'])['pa1'].transform('mean')
df['pa2_prom'] = df.groupby(['curso'])['pa2'].transform('mean')
df['final_prom'] = df.groupby(['curso'])['Final'].transform('mean')

In [4]:
################################################
#
# edad: asigno una categoría según el bin en un histograma por año-cuatrimestre
# ver: preparacion_dataset/03_grupos_dni.ipynb
#
# prom_edad: el promedio de la variable edad en el curso
#
################################################

labels = np.array([10,9,8,7,6,5,4,3,2,1])
for c in cuatrimestres:
    bins = np.histogram_bin_edges(df.loc[df['cuatrimestre']==c]['dni'])
    edades = pd.cut(x=df.loc[df['cuatrimestre']==c]['dni'], bins=bins, labels=labels, include_lowest=True)
    df.loc[df['cuatrimestre']==c, 'edad'] = edades

df = df.drop(['dni'], axis=1)
df['edad'] = df['edad'].values.astype(int)

df['sumedad'] = df.groupby(['curso'])['edad'].transform('sum')
df['prom_edad'] = df['sumedad']/df['n_alum']
df = df.drop(['sumedad'], axis=1)

In [5]:
################################################
#
# Condición
#
# Abandona1 : No rinde ambos parciales
# Abandona2 : Rinde parcial 1 pero no parcial 2
# Insuficiente : rinde ambos parciales y la suma no alcanza 8
# Examen : La suma está entre 8 y 12
# Promociona : La suma de ambos parciales es 13 o más.
#
################################################

df['sumaparciales'] = df['pa1'] + df['pa2']
df.loc[df['sumaparciales'] < 8, 'condición'] = 'Insuficiente'
df.loc[(df['sumaparciales'] > 7) & (df['sumaparciales'] < 13), 'condición'] = 'Examen'
df.loc[df['sumaparciales'] > 12, 'condición'] = 'Promociona'
df.loc[df['pa1'].isna(), 'condición'] = 'Abandona1'
df.loc[(df['pa1'].isna()) & (df['pa2'].notna()), 'condición'] = 'Insuficiente'
df.loc[(df['pa1'].notna()) & (df['pa2'].isna()), 'condición'] = 'Abandona2'
df = df.drop(['sumaparciales'], axis=1)
pd.concat([df[['pa1','pa2','Final','rem1','rem2','condición']].sample(10),
    df.loc[df['rem2'].notna(),['pa1','pa2','Final','rem1','rem2','condición']].sample(5)])

,pa1,pa2,Final,rem1,rem2,condición
177025,NaN,NaN,NaN,NaN,NaN,Abandona1
230460,1.0,NaN,NaN,NaN,NaN,Abandona2
84007,9.0,5.0,NaN,NaN,NaN,Promociona
77907,NaN,NaN,NaN,NaN,NaN,Abandona1
211232,2.0,3.0,NaN,NaN,NaN,Insuficiente
185252,10.0,7.0,NaN,NaN,NaN,Promociona
189153,NaN,NaN,NaN,NaN,NaN,Abandona1
203533,NaN,NaN,NaN,NaN,NaN,Abandona1
114506,NaN,NaN,NaN,NaN,NaN,Abandona1
70001,NaN,NaN,NaN,NaN,NaN,Abandona1


In [6]:
################################################
#
# abandona1_p, abandona2_p, valido1, valido2
#
# Detecto cursos donde todas las notas son NaN, esos
# cursos no son válidos, no se actualizaron sus notas.
#
################################################

df['temp1'] = (df['condición'] == 'Abandona1').astype(int)
df['temp1'] = df.groupby(['curso'])['temp1'].transform('sum')
df['abandona1_p'] = df['temp1'] / df['n_alum']
df['temp2'] = (df['condición'] == 'Abandona2').astype(int)
df['temp2'] = df.groupby(['curso'])['temp2'].transform('sum')
df.loc[df['abandona1_p'] == 1, 'temp2'] == np.nan
df['abandona2_p'] = df['temp2'] / (df['n_alum'] - df['temp1'])
df = df.drop(['temp1'], axis=1)
df = df.drop(['temp2'], axis=1)

# ¿Hay algún curso donde todas las notas de algún examen sean NaN?
cursos1 = df.loc[df['abandona1_p'] == 1, 'curso'].unique()
print("Cursos sin notas de parcial 1: ", len(cursos1))
# marco como válidos los cursos con alguna nota de parcial
df['valido1'] = (df['abandona1_p'] < 1).astype(int)
cursos2 = df.loc[df['abandona2_p'] == 1, 'curso'].unique()
print("Cursos sin notas de parcial 2: ", len(cursos2))
df['valido2'] = (df['abandona2_p'] < 1).astype(int)

Cursos sin notas de parcial 1:  799
Cursos sin notas de parcial 2:  102


In [7]:
# Corrijo un curso que tiene mal cargada la sede
df.loc[df['curso'] == 2376, 'SEDE'] = 1

df = df.drop(['cuatrimestre'], axis=1)
df.to_csv('../datos/dataset_02-feateng.csv', index=False)